Code modified from https://colab.research.google.com/github/justinbois/bootcamp/blob/gh-pages/2021/lessons/l40_serial.ipynb#scrollTo=EETrbqldzxsz

In [ ]:
%pip install pyserial

In [1]:
import serial
import serial.tools.list_ports
import numpy as np
import csv
import datetime
import pandas as pd

In [2]:
def find_arduino(port=None):
    """Get the name of the port that is connected to Arduino."""
    if port is None:
        ports = serial.tools.list_ports.comports()
        for p in ports:
            if p.manufacturer is not None and "Arduino" in p.manufacturer:
                port = p.device
    return port

def parse_line(line_in):
    return str.split(str.split(str(line_in),"\\r\\n")[0],"b'")[1]

In [3]:
#load arduino
ports = serial.tools.list_ports.comports()
port = find_arduino()
arduino = serial.Serial(port, baudrate=9600, timeout=1)
arduino.close()
arduino.open()

In [12]:
columns=[]
readings = 0
while readings<3: #how many beakers, can edit for some wiggle room
    line_in = arduino.read_until()
    if line_in == bytes('Next Measurement\r\n','utf-8'):
        print("next measurement")
        times = []
        vals = []
        count = 0
        while count < 5: #edit to change how many readings
            line_in = arduino.read_until()
            time = str(datetime.datetime.now())
            if line_in is not bytes():
                val = float(parse_line(line_in))
                times.append(time)
                vals.append(val)
                print(val)
                count = count+1
        columns.append(times)
        columns.append(vals)
        readings = readings+1 
    

next measurement
3299.571
3299.581
3299.587
3299.571
3299.587
next measurement
3299.5
3299.558
3299.581
3299.629
3299.69
next measurement
3299.648
3299.684
3299.629
3299.629
3299.568


In [14]:
pd.DataFrame(columns).T.to_csv("data.csv")